In [12]:
import rasterio
import numpy as np
import time
from sklearn.metrics import confusion_matrix, accuracy_score
from rasterio.warp import calculate_default_transform, reproject, Resampling


# Start the timer
start_time = time.time()
# Reclassification dictionaries for two rasters (replace with actual data)

non_active = [0,61,63,64,65,82,83,87,88,
                            111,112,121,122,123,124,131,
                            141,142,143,152,176,190,195]
cdl_values = {12: 1, 13: 1, 255: 1, 266: 1, 237: 1, 26: 5, 241:5, 254: 5, 238: 2, 239: 2, 23: 3}
all_values = [0,61,63,64,65,82,83,87,88,
                            111,112,121,122,123,124,131,
                            141,142,143,152,176,190,195,1,5,2,23,4]
range_min, range_max = 0, 240
# Paths to the original raster files
path_to_original_ground_truth = r"C:\Users\User\Documents\CS_sequence_check\kansas_area\kansas_area_predicted_raster.tif"
output_path = r"C:\Users\User\Documents\CS_sequence_check\kansas_area\kansas_area_predicted_raster_reclass.tif"

def reclassify_raster(input_path, output_path, cdl_values, non_active, all_values, range_min, range_max):
    try:
        with rasterio.open(input_path) as src:
            raster = src.read(1)  # Read the first band
            reclassified_raster = np.copy(raster)

            # Change values as per cdl_values dict, leave others as is
            for old_value, new_value in cdl_values.items():
                mask = raster == old_value
                reclassified_raster[mask] = new_value

            # Change specified values to 4, leave others as is
            for value in non_active:
                mask = raster == value
                reclassified_raster[mask] = 4

            # Set pixels within the range and not in the exclusion list to 6, leave others as is
            reclassify_mask = np.isin(reclassified_raster, all_values, invert=True) & \
                              (reclassified_raster >= range_min) & \
                              (reclassified_raster <= range_max)
            reclassified_raster[reclassify_mask] = 6

            new_meta = src.meta
            new_meta.update(dtype=rasterio.int32)

            with rasterio.open(output_path, 'w', **new_meta) as dst:
                dst.write(reclassified_raster, 1)
    except Exception as e:
        print(f"An error occurred: {e}")
# # # Reclassify both rasters
reclassify_raster(path_to_original_ground_truth, output_path, cdl_values, non_active, all_values, range_min, range_max)



# # Write the reclassified raster to a new file
# with rasterio.open(output_path, 'w', **new_meta) as dst:
#     dst.write(reclassify_raster, 1)